In [1]:
import sys
from time import time

from gensim.models.wrappers import FastText

In [2]:
category_list = [
    'Person', 'Location', 'Event', 'Organization', 'DateTime', 'PersonType', 'Currency', 'Nationality', 'Thing'
]

In [3]:
t = time()
# Load pre trained model
w2v_model = FastText.load_fasttext_format(model_file='models/fasttext/cc.tr.300.bin')

print('Time to load the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to load the model: 4.08 mins


In [9]:
import pandas as pd

t = time()

category_vectors = {}
for category in category_list:
    category_vectors[category] = []
    
# Read labeled words
df = pd.read_excel('data/labeled_words_single_v4.xlsx')

counter = 0
hit_word_count = 0
# Read Excel file and fill the dictionary
for row in df.index:
    word = str(df['Word'][row])
    label = df['Label'][row]
    
    if word in w2v_model.wv:
        hit_word_count += 1
        category_vectors[label].append(w2v_model.wv[word])
    
    counter += 1
    sys.stdout.write('\r'+str(counter))

print('Time to create vector array: {} mins'.format(round((time() - t) / 60, 2)))

1899

93765Time to create vector array: 0.34 mins


In [10]:
# print(category_vectors['Person'][0])
print('Missed ' + str(counter - hit_word_count) + ' from ' + str(counter))

Missed 490 from 93765


In [14]:
import numpy as np

t = time()

centroids = {}

for category in category_list:
    centroids[category] = np.sum(category_vectors[category], axis=0) / len(category_vectors[category])
    
print('Time to calculate centroids: {} mins'.format(round((time() - t) / 60, 2)))

Time to calculate centroids: 0.0 mins


In [16]:
#print(category_vectors['Person'][0])  # -5.33770142e+02  --- -9.28570423e-03
#print(len(category_vectors['Person']))
#print(centroids['Person'])

In [17]:
import pandas as pd

from numpy import dot
from numpy.linalg import norm

t = time()

# features will be stored in this dataframe
output_df = pd.DataFrame()

# Read labeled words
df = pd.read_excel('data/labeled_words_single_v4.xlsx')

counter = 0

# Read Excel file and fill the dictionary
for row in df.index:
    word = str(df['Word'][row])
    label = str(df['Label'][row])

    if word in w2v_model.wv:
        
        output = { 'Word': word, 'Label': label  }
        
        # calculate distance to each centroid
        for category in category_list:
            output[category] = dot(w2v_model.wv[word], centroids[category])/(norm(w2v_model.wv[word])*norm(centroids[category]))
            #output[category] = 1 - spatial.distance.cosine(w2v_model.wv[word],centroids[category])
            
        output_df = output_df.append(output, ignore_index=True)
        
    counter += 1
    #if counter == 1000:
        #break
    sys.stdout.write('\r'+str(counter))

# df["Person"] = pd.to_numeric(df["Person"])
print('Time to calculate centroid distances categories: {} mins'.format(round((time() - t) / 60, 2)))
output_df.head(5)
# print('Missed ' + str(counter - hit_word_count) + ' from ' + str(counter))

93765Time to calculate centroid distances categories: 42.28 mins


,Currency,DateTime,Event,Label,Location,Nationality,Organization,Person,PersonType,Thing,Word
0,0.250148,0.122472,0.338534,Person,0.560342,0.338208,0.474824,0.652741,0.333379,0.365368,Corina
1,0.162727,0.191302,0.288349,Person,0.351361,0.222733,0.320877,0.431188,0.316707,0.320074,Casanova
2,0.258475,0.158841,0.253603,Location,0.217465,0.160455,0.288744,0.165113,0.136111,0.167267,İsviçre
3,0.205449,0.221836,0.289384,PersonType,0.288628,0.253223,0.291755,0.256634,0.286041,0.322561,Şansölyesidir
4,0.172876,0.070771,0.125732,PersonType,0.175117,0.220059,0.192620,0.253646,0.467836,0.173848,avukat


In [18]:
t = time()

writer = pd.ExcelWriter('data/centroid_distances_v4.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
output_df.to_excel(writer, sheet_name='Sheet1', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()
print('Time to save: {} mins'.format(round((time() - t) / 60, 4)))

Time to save: 0.5978 mins


In [32]:
output_df.loc[899]

Currency         0.226872
DateTime        0.0928929
Event            0.314503
Label              Person
Location         0.401173
Nationality      0.264712
Organization     0.345497
Person           0.446384
PersonType       0.274834
Thing            0.323062
Word               Kruger
Name: 899, dtype: object